In [1]:
import os
import torch
import sys
sys.path.append("..")
from utils.misc import create_dir, create_log_dir, load_checkpoint, save_checkpoint, tab_printer, Params, set_logger
from utils.gelu import GELU
from generate.generate_images import generate_gif

In [ ]:
import tensorflow as tf

In [2]:
torch.cuda.is_available()

True

In [3]:
torch.cuda.get_device_name(0)

'GeForce RTX 2070'

In [ ]:
img = torch.randn(1, 1, 28, 28)

In [ ]:
img.size(0)

In [4]:
import torch.nn as nn

In [5]:
def conv_out_shape(w, k, s, p=0):
    return (w - k + 2 * p) / s + 1

In [6]:
def tconv_out_shape(w, k, s):
    return w * s + max(k - s, 0)

In [ ]:
tconv_out_shape(4, 3, 2)

In [ ]:
tconv_out_shape(9, 3, 2)

In [ ]:
tconv_out_shape(19, 2, 2)

In [ ]:
tconv_out_shape(27, 2, 1)

In [ ]:
[4, 3, 2]

In [ ]:
conv_out_shape(112, 4, 2)

In [ ]:
conv_out_shape(13, 4, 2)

In [ ]:
conv_out_shape(5, 4, 2)

In [ ]:
conv_out_shape(1, 4, 2)

In [8]:
class Flatten(nn.Module):
    def forward(self, x):
        return x.view(x.size(0), -1)

In [ ]:
flatten = Flatten()

In [ ]:
f = flatten(x2)

In [ ]:
f.shape

In [7]:
class UnFlatten(nn.Module):
    def __init__(self, channels, height, width):
        super(UnFlatten, self).__init__()
        self.channels = channels
        self.height = height
        self.width = width

    def forward(self, x):
        return x.view(x.size(0), self.channels, self.height, self.width)

In [86]:
conv_layers = [
            nn.Conv2d(in_channels=1,
                      out_channels=64,
                      kernel_size=4,
                      stride=2),
            nn.BatchNorm2d(64),
            GELU(),
            nn.Conv2d(in_channels=64,
                      out_channels=128,
                      kernel_size=4,
                      stride=2),
            nn.BatchNorm2d(128),
            GELU(),
            nn.Conv2d(in_channels=128,
                      out_channels=256,
                      kernel_size=4,
                      stride=2),
            nn.BatchNorm2d(256),
            GELU(),
            Flatten(),
            nn.Linear(in_features=27648, out_features=32)
        ]

In [87]:
conv_encoding = nn.Sequential(*conv_layers)

In [88]:
i = conv_encoding(img)

In [89]:
92*112

10304

In [90]:
i.shape # 9 12 -> 92 112

torch.Size([1, 32])

In [93]:
tconv_layers = [
            nn.Linear(in_features=32, out_features=27648),
            UnFlatten(256, 9, 12),
            nn.ConvTranspose2d(in_channels=256,
                               out_channels=128,
                               kernel_size=(5, 4),
                               stride=2),
            nn.BatchNorm2d(128),
            GELU(),
            nn.ConvTranspose2d(in_channels=128,
                               out_channels=64,
                               kernel_size=(4, 4),
                               stride=2),
            nn.BatchNorm2d(64),
            GELU(),
            nn.ConvTranspose2d(in_channels=64,
                               out_channels=1,
                               kernel_size=(6, 6),
                               stride=2),
            nn.Sigmoid(),
        ]

decoding = nn.Sequential(*tconv_layers)

In [94]:
decoding(i).shape

torch.Size([1, 1, 92, 112])

In [58]:
tconv_out_shape(9, 5, 2)

21

In [61]:
tconv_out_shape(21, 4, 2)

44

In [62]:
tconv_out_shape(44, 6, 2)

92

In [68]:
tconv_out_shape(12, 4, 2)

26

In [71]:
tconv_out_shape(26, 4, 2)

54

In [72]:
tconv_out_shape(54, 6, 2)

112

In [ ]:
fc_mu = nn.Linear(in_features=4096, out_features=50)

In [ ]:
fc_mu(i)

In [ ]:
i.shape

In [ ]:
torch.Size([1, 4096])

In [ ]:
import gym

In [ ]:
torch.randn(torch.Size([1, 4096])).shape

In [ ]:
root = os.path.abspath('..')
model_dir = os.path.join(root, 'experiments/vae')
json_path = os.path.join(model_dir, 'params.json')

In [ ]:
json_path

In [ ]:
params = Params(json_path)

In [ ]:
params

In [ ]:
tab_printer(params)

In [ ]:
generate_gif('/home/aktersnurra/Documents/Projects/variational-autoencoders/experiments/vae_w_bn/runs/2019-08-07_23-42-01/generated_images')

In [12]:
img = torch.randn(1, 1, 92, 112)